In [8]:
import os
import json
import time
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


LOGIN_URL = "https://tmslog.com.br/"  
USERNAME = "sislogica@sislogica.com.br"  
PASSWORD = "S1s@2023"  
DASHBOARD_URL = "https://tmslog.com.br/inicial.aspx" 


def login_with_selenium(login_url, username, password, protected_url):
    """
    Faz login usando Selenium e retorna o HTML da página protegida.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(login_url)

    time.sleep(3)  

    driver.find_element(By.NAME, "usuario").send_keys(username)  
    driver.find_element(By.NAME, "senha").send_keys(password) 
    driver.find_element(By.NAME, "senha").send_keys(Keys.RETURN) 

    time.sleep(5) 

    driver.get(protected_url)
    time.sleep(3) 

    html = driver.page_source
    driver.quit()

    return html


class Website:
    """
    Classe para representar um Website que foi webscraped, incluindo suporte a login.
    """

    def __init__(self, url, html=None):
        self.url = url
        self.body = html if html else requests.get(url, headers=headers).content

        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


def get_all_details(url, html):
    """
    obtem todos os detalhes de um site, incluindo a area autenticada
    """
    result = "Landing page:\n"
    result += Website(url, html).get_contents()
    return result


def translate_to_portuguese(text):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Você é um tradutor profissional de inglês para português. Traduza o texto mantendo o mesmo formato e preservando as marcações markdown."},
            {"role": "user", "content": f"Traduza o seguinte texto para português:\n\n{text}"}
        ],
        response_format={"type": "text"}
    )
    return response.choices[0].message.content


if __name__ == "__main__":
    print("Iniciando login e scraping...")

    html_protegido = login_with_selenium(LOGIN_URL, USERNAME, PASSWORD, DASHBOARD_URL)

    detalhes_site = get_all_details(DASHBOARD_URL, html_protegido)

    detalhes_traduzidos = translate_to_portuguese(detalhes_site)

    print("\n✅ Conteúdo traduzido da área autenticada:\n")
    print(detalhes_traduzidos)


API key looks good so far
🔥 Iniciando login e scraping...

✅ Conteúdo traduzido da área autenticada:

**Landing page:**
**Webpage Title:**
Nenhum  
**Webpage Contents:**
InfinityMbps  
Relatórios  
Exibir todos os Relatórios  
Chamados  
Exibir todos os Chamados  
Posso ajudar?  
Tópico de tutoriais  
Sislogica ADM  
Olá, Sislogica ADM!  
Meus Dados  
Trocar Senha  
Configurações  
Idiomas  
Ver Original  
Selecione o idioma  
Abecásio  
Achinês  
Acholi  
Afar  
Africâner  
Aimará  
Albanês  
Alemão  
Alur  
Amárico  
Andebele (meridional)  
Árabe  
Armênio  
Assamês  
Avar  
Awadhi  
Azerbaijano  
Balinês  
Balúchi  
Bambara  
Basco  
Bashkir  
Batak karo  
Batak simalungun  
Batak toba  
Baúle  
Bemba  
Bengali  
Betawi  
Bielorrusso  
Bikol  
Birmanês  
Boiapuri  
Bósnio  
Bretão  
Búlgaro  
Buriata  
Canarês  
Cantonês  
Catalão  
Cazaque  
Cebuano  
Chamorro  
Checheno  
Chicheua  
Chinês (simplificado)  
Chinês (tradicional)  
Chona  
Chuquês  
Chuvache  
Cingalês  
Cómi  
Conca

In [7]:
!pip install requests beautifulsoup4 selenium python-dotenv openai ipython webdriver-manager


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------  9.4/9.5 MB 58.4 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 39.5 MB/s eta 0:00:00
